In [1]:
import numpy as np

kpts_A_sph = np.load("/data/code/glue-factory/datasets/spherecraft_data/barbershop/features_xfeat_spherical/00000000.npz")['keypoints']
kpts_B_sph = np.load("/data/code/glue-factory/datasets/spherecraft_data/barbershop/features_xfeat_spherical/00000001.npz")['keypoints']

print(kpts_A_sph[:5])
print(f"Phi range: [{kpts_A_sph[:, 0].min(), kpts_A_sph[:, 0].max()}] ---- [-pi, pi] (longitude = azimuth)")
print(f"Theta range: [{kpts_A_sph[:, 1].min(), kpts_A_sph[:, 1].max()}] ---- [-pi/2, pi/2] (latitude = polar)")

[[-2.282797    0.4095405 ]
 [ 0.5467267   0.19958098]
 [ 1.6663945  -0.47036964]
 [ 1.3681377   0.04406013]
 [ 1.7583883  -0.41795668]]
Phi range: [(np.float32(-3.1396358), np.float32(3.1389818))] ---- [-pi, pi] (longitude = azimuth)
Theta range: [(np.float32(-1.5491846), np.float32(1.0889592))] ---- [-pi/2, pi/2] (latitude = polar)


In [ ]:
def standard_spherical_to_pixel(kpts_sph_np, W, H):
    phi = kpts_sph_np[:, 0]
    theta = kpts_sph_np[:, 1]
    px = (phi / (2 * np.pi) + 0.5) * (W - 1) - 0.5
    py = (-theta / np.pi + 0.5) * (H - 1) - 0.5
    return np.stack([px, py], axis=-1)

def pixel_to_spherical_Z_UP(kpts_px, W, H):
    px = kpts_px[:, 0]
    py = kpts_px[:, 1]
    
    theta = (px + 0.5) / (W - 1) * (2 * np.pi)  # azimuthal [0, 2*pi]
    phi = (py + 0.5) / (H - 1) * np.pi          # polar [0, pi]
    
    return np.stack([theta, phi], axis=-1)

def convert_from_normal_to_spherecraft_coordinates(kpts_sph_normal, W, H):
    """
    Convert spherical coordinates from Y-up system to Z-up system.
        1. calculate equirectangular pixel coordinates from spherical (Y-up)
        2. calculate spherical (Z-up) from equirectangular pixel coordinates 
    
    Parameters:
    - kpts_sph_y_up: Nx2 array of spherical coordinates (phi, theta) in Y-up system.
    
    Returns:
    - Nx2 array of spherical coordinates (theta, phi) in Z-up system.
    """

    px, py = standard_spherical_to_pixel(kpts_sph_normal, W, H)

    kpts_sph_z_up = pixel_to_spherical_Z_UP(np.stack([px, py], axis=-1), W, H)

    return kpts_sph_z_up

In [ ]:
def convert_y_up_to_z_up_spherical(kpts_sph_y_up, W, H):
    """
    Convert spherical coordinates from Y-up system to Z-up system.
    
    Y-up system: phi=azimuthal [-pi, pi], theta=polar [-pi/2, pi/2]
    Z-up system: theta=azimuthal [0, 2*pi], phi=polar [0, pi]
    
    Args:
        kpts_sph_y_up: Array of shape (N, 2) with [phi, theta] in Y-up system
    
    Returns:
        Array of shape (N, 2) with [theta, phi] in Z-up system
    """
    phi_y_up = kpts_sph_y_up[:, 0]  # azimuthal [-pi, pi]
    theta_y_up = kpts_sph_y_up[:, 1]  # polar [-pi/2, pi/2]
    
    # Convert to Cartesian in Y-up system
    x_y = np.cos(theta_y_up) * np.sin(phi_y_up)
    y_y = np.sin(theta_y_up)
    z_y = np.cos(theta_y_up) * np.cos(phi_y_up)
    
    # Transform coordinates: x_y->z_z, y_y->x_z, z_y->y_z - rotation
    # plus translation from center to bottom left
    x_z = z_y
    y_z = x_y - 1.0
    z_z = y_y - 1.0 
    
    # Convert to spherical in Z-up system
    d = np.sqrt(x_z**2 + y_z**2 + z_z**2)
    d_safe = np.clip(d, a_min=1e-8, a_max=None)
    
    theta_z_up = np.arctan2(y_z, x_z)  # azimuthal [-pi, pi]
    # theta_z_up = (theta_z_up + np.pi) # azimuthal [0, 2*pi]
    phi_z_up = np.arccos(np.clip(z_z/d_safe, a_min=-1.0, a_max=1.0))  # polar [0, pi]
    
    return np.stack([theta_z_up, phi_z_up], axis=-1)

In [20]:
def z_up_spherical_to_pixel(kpts_sph_z_up, W, H):
    """
    Convert Z-up spherical coordinates to equirectangular pixel coordinates.
    
    Args:
        kpts_sph_z_up: Array of shape (N, 2) with [theta, phi] in Z-up system
                      theta: azimuthal angle [-pi, pi] 
                      phi: polar angle [0, pi]
        W, H: Image width and height
    
    Returns:
        Array of shape (N, 2) with [px, py] pixel coordinates
    """
    theta = kpts_sph_z_up[:, 0]  # azimuthal [-pi, pi]
    phi = kpts_sph_z_up[:, 1]    # polar [0, pi]
    
    # Convert to equirectangular coordinates
    # theta [0, 2*pi] -> x [0, W]
    px = (theta / (2*np.pi)) * (W - 1) - 0.5
    
    # phi [0, pi] -> y [0, H] (note: phi=0 is north pole, should map to y=0)
    py = (phi / np.pi) * (H - 1) - 0.5
    
    return np.stack([px, py], axis=-1)


In [21]:
"""Test that the coordinate conversion preserves equirectangular pixel locations."""

# Create some test points in Y-up spherical coordinates
test_points_y_up = np.array([
    [0.0, 0.0],           # Front center
    [np.pi/2, 0.0],       # Right center
    [-np.pi/2, 0.0],      # Left center
    [np.pi, 0.0],         # Back center
    [0.0, np.pi/4],       # Front up
    [0.0, -np.pi/4],      # Front down
    [-np.pi, -np.pi/2],   # Bottom Left
    [0, -np.pi/2],        # Top center
])

W, H = 800, 400

# Convert using original Y-up system
def standard_spherical_to_pixel(kpts_sph_np, W, H):
    phi = kpts_sph_np[:, 0]
    theta = kpts_sph_np[:, 1]
    px = (phi / (2 * np.pi) + 0.5) * (W - 1) - 0.5
    py = (-theta / np.pi + 0.5) * (H - 1) - 0.5
    return np.stack([px, py], axis=-1)

pixels_original = standard_spherical_to_pixel(test_points_y_up, W, H)

# Convert to Z-up system and then to pixels
test_points_z_up = convert_y_up_to_z_up_spherical(test_points_y_up, W, H)
pixels_converted = z_up_spherical_to_pixel(test_points_z_up, W, H)

print("Original Y-up spherical coordinates:")
print(test_points_y_up)
print("\nConverted Z-up spherical coordinates:")
print(test_points_z_up)
print("\nPixel coordinates from Y-up system:")
print(pixels_original)
print("\nPixel coordinates from Z-up system:")
print(pixels_converted)
print("\nMax difference in pixel coordinates:")
print(np.max(np.abs(pixels_original - pixels_converted)))

[-0.78539816  0.         -1.57079633 -2.35619449 -0.95531662 -0.95531662
 -1.57079633 -1.57079633]
Original Y-up spherical coordinates:
[[ 0.          0.        ]
 [ 1.57079633  0.        ]
 [-1.57079633  0.        ]
 [ 3.14159265  0.        ]
 [ 0.          0.78539816]
 [ 0.         -0.78539816]
 [-3.14159265 -1.57079633]
 [ 0.         -1.57079633]]

Converted Z-up spherical coordinates:
[[-0.78539816  2.18627604]
 [ 0.          3.14159265]
 [-1.57079633  2.03444394]
 [-2.35619449  2.18627604]
 [-0.95531662  1.80553396]
 [-0.95531662  2.51925834]
 [-1.57079633  2.67794504]
 [-1.57079633  2.67794504]]

Pixel coordinates from Y-up system:
[[ 3.9900e+02  1.9900e+02]
 [ 5.9875e+02  1.9900e+02]
 [ 1.9925e+02  1.9900e+02]
 [ 7.9850e+02  1.9900e+02]
 [ 3.9900e+02  9.9250e+01]
 [ 3.9900e+02  2.9875e+02]
 [-5.0000e-01  3.9850e+02]
 [ 3.9900e+02  3.9850e+02]]

Pixel coordinates from Z-up system:
[[-100.375       277.16939713]
 [  -0.5         398.5       ]
 [-200.25        257.88586344]
 [-300.